In [1]:
import rainbow.geometry.volume_mesh as VM
import rainbow.geometry.volume_mesh_wildmesh as WM
import rainbow.geometry.volume_mesh_tetgen as TG
import rainbow.geometry.volume_mesh_sizing_functions as SF

import rainbow.geometry.mean_curvature as MC
import rainbow.geometry.surface_mesh as SURF
import rainbow.geometry.grid3 as GRID3

import numpy as np

ModuleNotFoundError: No module named 'rainbow.geometry.volume_mesh_tetgen'

In [2]:
def make_cut_plane_view(V, T, s=0.5, ax=0, colors=None):
    """
    Visualize tetrahedral mesh cut by a view-plane

    :param V:   Vertex array of mesh to be visualized
    :param T:   Tetrahedron array of mesh to be visualized
    :param ax:  The axis direction to create a orthogonal cut view plane on. ax=0, ax=1 or ax=2.
    :param s:   The position of the cut-plane along the axis ax. If zero everything is remove, if 1 then all is shown.
    :param colors: optional array of colors for the visualization
    :return:
    """
    import numpy as np
    import igl
    import meshplot as mp 
    min_corner = np.amin(V, axis=0)
    max_corner = np.amax(V, axis=0)
    diagonal = max_corner - min_corner
    cut_position = s * diagonal + min_corner
    c = igl.barycenter(V, T)
    idx = np.where(c[:, ax] < cut_position[ax])
    if colors is not None:
        mp.plot(V, T[idx[0], :], colors, shading={"wireframe": True})
    else:
        mp.plot(V, T[idx[0], :], shading={"wireframe": True})

In [ ]:
# Create Two Simple Shaped a Box and a Sphere
boxV, boxF = SURF.create_box(1, 1, 1)
sphereV, sphereF = SURF.create_sphere(0.5, 25, 25)

In [ ]:
# Create background mesh with sizing function
I = 4
J = 4
K = 4
min_value = 1/40
max_value = 1/10
box_grid = SF.create_linear_field_around_mesh(boxV, I, J, K, min_value, max_value)

# Show inside of the background mesh
GRID3.show_layer(box_grid, 1)

# Get the volume mesh data
bV, bT, bA = VM.from_grid3d(box_grid)

In [ ]:
# Create background mesh with sizing function
sphere_grid = GRID3.create_signed_distance(sphereV, sphereF, 20, 20, 20)
sphere_grid = MC.create_from_sdf(sphere_grid)

## Use linear iterpolation
emin = 1/80
emax = 1/10
kmin = min(sphere_grid.values)
kmax = max(sphere_grid.values)

for k in range(len(sphere_grid.values)):
    # Linear interpolation
    S = max(min(sphere_grid.values[k],kmax),kmin)/(kmax-kmin)
    sphere_grid.values[k] = emin * (1 - S) + emax * S
    
# Show inside of the background mesh
GRID3.show_layer(sphere_grid, 10)

# Get the volume mesh data
sV, sT, sA = VM.from_grid3d(sphere_grid)

In [ ]:
WM_options = WM.Options()
# Box
V3, T3 = WM.create_mesh(boxV, boxF, WM_options, bV, bT, bA)

# Sphere
V4, T4 = WM.create_mesh(sphereV, sphereF, WM_options, sV, sT, sA)

make_cut_plane_view(V3, T3, ax=2)
make_cut_plane_view(V4, T4, ax=2)

In [ ]:
TG_options = TG.Options()
# Box
V1, T1 = TG.create_mesh(boxV, boxF, TG_options, bV, bT, bA)

# Sphere
V2, T2 = TG.create_mesh(sphereV, sphereF, TG_options, sV, sT, sA)

make_cut_plane_view(V1, T1, d=2)
make_cut_plane_view(V2, T2, d=2)

# New additions to the sizing function interface

In [ ]:
V_box, F_box = SURF.create_box(10,10,10)
grid = GRID3.Grid(np.array([-10,-10,-10]), np.array([10,10,10]), 20, 20, 20)

In [ ]:
phi = SF.signed_distance_field(V_box, F_box)
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), phi)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
h = SF.scale(phi, SF.const(2))
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), h)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), SF.absolute(phi))
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
h = SF.clamp(phi, SF.const(-3), SF.const(3))
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), h)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
kappa = SF.curvature(phi)
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), kappa)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
h = SF.if_statement( SF.greater(phi, SF.const(0)), SF.const(2), SF.const(1) )
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), h)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
h = SF.if_statement( SF.less(phi, SF.const(0)), SF.const(2), SF.const(1) )
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), h)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)

In [ ]:
h = SF.if_statement( SF.greater(phi, SF.const(0)), phi, SF.add(phi, SF.const(5)) )
grid.values = SF.sample_vertices(GRID3.get_nodes_array(grid), h)
bV, bT, bA = VM.from_grid3d(grid)
make_cut_plane_view(bV, bT, ax=2, colors=bA)